In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import lxml.html
import requests
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import time
import math
import re
import json
import plotly.graph_objects as go
import plotly.express as px
from bs4 import BeautifulSoup as bs4
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta

In [2]:
team_list = ['Lima Time!', 'Mom\'s Cookin', 'Harveys Wallbangers', 'Roid Ragers', 'Charmer', 'Wisconsin Wu-Tang', 'Midnight Madness', 'Brewbirds', 'Dirty Birds', 'Trouble With The Curve', 'Lil Trump &amp;amp; the Ivanabees', 'Young Guns']
stat_list = ['BA', 'HR', 'R', 'RBI', 'SB', 'ERA', 'HDS', 'K', 'W', 'WHIP']

In [3]:
driver = webdriver.Chrome("C:\\WebDriver\\bin\\chromedriver.exe")

In [7]:
driver.get('https://www.cbssports.com/login?master_product=150&xurl=https%3A%2F%2Fwww.cbssports.com%2Flogin')
search_box = driver.find_element_by_id('userid')
search_box.send_keys('gostros09')
search_box = driver.find_element_by_id('password')
time.sleep(1)
search_box.send_keys('Segneri9A')
time.sleep(1)
search_box.submit()
time.sleep(2)
driver.get('https://xdl.baseball.cbssports.com/teams/38')
time.sleep(2) # Let the user actually see something!
driver.find_element(By.XPATH, "//*[text()='Projections']").click()
#driver.quit()

In [ ]:
def get_week_data(wk):
    driver.get('https://xdl.baseball.cbssports.com/standings/overall/'+str(wk))
    time.sleep(2)
    html = driver.page_source
    soup = bs4(html, 'html.parser')

    # Locate hitting tables
    df = pd.read_html(str(soup.select_one('tr.label').find_next('table').find_next('table')))[0]
    df.columns=['Team', 'Stat', 'Pts', 'Dif']
    df = df.iloc[1:,:]

    ba = df.iloc[1:13]
    ba.columns=['Team', 'ba', 'Pts', 'Dif']
    hr = df.iloc[14:26]
    hr.columns=['Team', 'hr', 'Pts', 'Dif']
    r = df.iloc[27:39]
    r.columns=['Team', 'r', 'Pts', 'Dif']
    rbi = df.iloc[40:52]
    rbi.columns=['Team', 'rbi', 'Pts', 'Dif']
    sb = df.iloc[53:65]
    sb.columns=['Team', 'sb', 'Pts', 'Dif']

    # Locate pitching tables
    df2 = pd.read_html(str(soup.select_one('tr.label').find_next('table').find_next('table').find_next('table')))[0]
    df2.columns=['Team', 'Stat', 'Pts', 'Dif']
    df2 = df2.iloc[1:,:]

    era = df2.iloc[1:13]
    era.columns=['Team', 'era', 'Pts', 'Dif']
    hds = df2.iloc[14:26]
    hds.columns=['Team', 'hds', 'Pts', 'Dif']
    k = df2.iloc[27:39]
    k.columns=['Team', 'k', 'Pts', 'Dif']
    w = df2.iloc[40:52]
    w.columns=['Team', 'w', 'Pts', 'Dif']
    whip = df2.iloc[53:65]
    whip.columns=['Team', 'whip', 'Pts', 'Dif']
    
    df = [ba, hr, r, rbi, sb, era, hds, k, w, whip]
    return df

In [234]:
def parse_name(n):
    n = n.split()
    return ' '.join(n[:n.index('|')-1])

def parse_pos(p):
    p = p.split()
    return p[p.index('|')-1]

def parse_team(t):
    t = t.split()
    return t[-1]

def str_2_int(x):
    try:
        return int(x)
    except:
        return x

In [267]:
teams_dict = {'Brewbirds':'3', 'Charmer':'36', 'Dirty Birds':'41', 'Harvey':'4', 'Lil Trump':'27', 'Lima Time':'38', 'Midnight':'30', 'Ugly Spuds':'29', 'Roiders':'44', 'Trouble':'1', 'Wu-Tang':'42', 'Young Guns':'45'}
teams_df = pd.DataFrame(teams_dict,index=['team_num']).T

In [288]:
html = driver.page_source
soup = bs4(html, 'html.parser')

In [343]:
def get_team_projections(owner):
    b = pd.read_html(str(soup.find_all('table')[1]), header=1, converters={'AB':str_2_int,'H':str_2_int, 'R':str_2_int,'HR':str_2_int,'RBI':str_2_int,'SB':str_2_int})[0].drop(columns='Edit')
    b = b[b['Pos']!='Reserves'].reset_index(drop=False).drop(columns='index')
    b.loc[b.index<14, 'Status'] = 'Start'
    b.loc[b.index>=14, 'Status'] = 'Bench'
    b['Name'] = b.Players.apply(lambda x: parse_name(x))
    b['Elig'] = b.Players.apply(lambda x: parse_pos(x))
    b['Team'] = b.Players.apply(lambda x: parse_team(x))
    b['Owner'] = owner
    
    p = pd.read_html(str(soup.find_all('table')[2]), header=1, converters={'IP':str_2_int,'BBI':str_2_int, 'HA':str_2_int, 'W':str_2_int,'S':str_2_int,'K':str_2_int})[0].drop(columns='Edit')
    p = p[~p['Pos'].str.contains('Reserve')].reset_index(drop=False).drop(columns='index')
    p['ER'] = (p['ERA'].astype(float)/9)*p['INNs'].astype(float)
    p.loc[p.index<9, 'Status'] = 'Start'
    p.loc[p.index>=9, 'Status'] = 'Bench'
    p['Name'] = p.Players.apply(lambda x: parse_name(x))
    p['Elig'] = p.Players.apply(lambda x: parse_pos(x))
    p['Team'] = p.Players.apply(lambda x: parse_team(x))
    p['Owner'] = owner
    return b, p

In [301]:
driver.get('https://xdl.baseball.cbssports.com/teams/'+num)
driver.find_element(By.XPATH, "//*[text()='Projections']").click()
time.sleep(3)
html = driver.page_source
soup = bs4(html, 'html.parser')

In [302]:
b = pd.read_html(str(soup.find_all('table')[1]), header=1, converters={'AB':str_2_int,'R':str_2_int,'HR':str_2_int,'RBI':str_2_int,'SB':str_2_int})[0]#.drop(columns='Edit')
b

,Edit,Pos,Players,AB,R,H,1B,2B,3B,HR,RBI,BB,KO,SB,CS,BA,OBP,SLG,Roto Rank
0,NaN,C,"Tyler Stephenson 1B,C | CIN",373,50,95,67,19,0,9,41,43,96,1,0,.255,.337,.378,187
1,NaN,1B,"Jared Walsh 1B,OF | LAA",557,76,153,87,32,2,32,101,59,153,2,1,.275,.346,.512,161
2,NaN,2B,"Whit Merrifield 2B,OF | KC",618,96,173,119,37,3,14,78,39,100,35,5,.280,.324,.417,35
3,NaN,3B,"Gio Urshela 3B,SS | NYY",534,67,147,97,26,4,20,75,40,131,2,0,.275,.327,.451,295
4,NaN,SS,Carlos Correa SS | HOU,547,95,153,94,32,2,25,86,64,112,2,0,.280,.358,.483,78
5,NaN,MI,"Gleyber Torres 2B,SS | NYY",565,72,155,110,29,1,15,71,61,127,16,6,.274,.346,.409,222
6,NaN,CI,Jose Abreu 1B | CHW,549,86,155,88,31,2,34,117,56,134,1,0,.282,.365,.532,67
7,NaN,OF,Harrison Bader OF | STL,444,60,119,70,27,2,20,65,36,104,11,4,.268,.332,.473,199
8,NaN,OF,Starling Marte OF | NYM,596,118,193,132,36,6,19,79,53,117,54,8,.324,.394,.500,27
9,NaN,OF,Austin Meadows OF | TB,546,89,138,71,31,4,32,114,60,131,6,4,.253,.332,.500,118


In [315]:
b = pd.DataFrame()
p = pd.DataFrame()
for owner, num in teams_dict.items():
    driver.get('https://xdl.baseball.cbssports.com/teams/'+num)
    driver.find_element(By.XPATH, "//*[text()='Projections']").click()
    time.sleep(4)
    html = driver.page_source
    soup = bs4(html, 'html.parser')
    result = get_team_projections(owner)
    b = b.append(result[0])
    p = p.append(result[1])

In [323]:
df = b[b['Status']=='Start'].groupby('Owner').agg({'R':'sum', 'RBI':'sum', 'HR':'sum', 'SB':'sum', 'H':'sum', 'AB':'sum'})
df['BA'] = df['H']/df['AB']
df

,R,RBI,HR,SB,H,AB,BA
Owner,,,,,,,
Brewbirds,1158,1158,322,158,2086,7501,0.278096
Charmer,1185,1147,380,161,1973,7267,0.271501
Dirty Birds,1138,1143,351,88,2105,7402,0.284383
Harvey,1205,1146,392,82,2005,7391,0.271276
Lil Trump,1132,1129,352,142,2030,7291,0.278425
Lima Time,1145,1188,379,153,1963,7067,0.277770
Midnight,1052,985,329,74,1709,6500,0.262923
Roiders,1261,1254,415,110,2103,7574,0.277660
Trouble,1187,1153,359,109,2092,7690,0.272042


In [355]:
b_ranks = df.drop(columns=['H', 'AB']).rank()
b_ranks['Pts'] = b_ranks.sum(axis=1)
b_ranks

,R,RBI,HR,SB,BA,Pts
Owner,,,,,,
Brewbirds,8.0,10.0,2.0,11.0,9.0,40.0
Charmer,9.0,7.5,10.0,12.0,5.0,43.5
Dirty Birds,5.0,5.0,5.0,3.0,12.0,30.0
Harvey,11.0,6.0,11.0,2.0,4.0,34.0
Lil Trump,4.0,4.0,6.0,9.0,10.0,33.0
Lima Time,7.0,11.0,9.0,10.0,8.0,45.0
Midnight,1.0,1.0,4.0,1.0,2.0,9.0
Roiders,12.0,12.0,12.0,6.0,7.0,49.0
Trouble,10.0,9.0,7.0,5.0,6.0,37.0


In [357]:
dfp = p[p['Status']=='Start'].groupby('Owner').agg({'W':'sum', 'K':'sum', 'S':'sum', 'INNs':'sum', 'ER':'sum', 'HA':'sum', 'BBI':'sum'})
dfp['ERA'] = (dfp['ER']/dfp['INNs'])*9
dfp['WHIP'] = (dfp['HA']+dfp['BBI'])/dfp['INNs']
p_ranks = dfp.drop(columns=['ER', 'HA', 'BBI']).rank()
p_ranks['Pts'] = p_ranks.sum(axis=1)
p_ranks

,W,K,S,INNs,ERA,WHIP,Pts
Owner,,,,,,,
Brewbirds,11.0,10.0,8.0,7.0,10.0,7.0,53.0
Charmer,2.5,2.0,10.0,3.0,5.0,11.0,33.5
Dirty Birds,7.0,3.0,12.0,2.0,1.0,3.0,28.0
Harvey,9.5,8.0,1.5,11.0,7.0,6.0,43.0
Lil Trump,8.0,9.0,1.5,10.0,6.0,4.0,38.5
Lima Time,4.0,7.0,3.0,9.0,12.0,12.0,47.0
Midnight,2.5,4.5,9.0,5.0,3.0,1.0,25.0
Roiders,1.0,1.0,7.0,1.0,8.0,9.0,27.0
Trouble,5.5,6.0,11.0,6.0,2.0,2.0,32.5


In [359]:
(b_ranks['Pts']+p_ranks['Pts']).sort_values()

Owner
Midnight       34.0
Ugly Spuds     44.0
Dirty Birds    58.0
Trouble        69.5
Lil Trump      71.5
Roiders        76.0
Charmer        77.0
Harvey         77.0
Wu-Tang        78.5
Young Guns     87.5
Lima Time      92.0
Brewbirds      93.0
Name: Pts, dtype: float64

In [364]:
dfp

,W,K,S,INNs,ER,HA,BBI,ERA,WHIP
Owner,,,,,,,,,
Brewbirds,84,1301,74,1210,481.034444,1017,388,3.577942,1.161157
Charmer,70,1053,105,1092,414.297778,986,326,3.414542,1.201465
Dirty Birds,76,1066,122,1057,369.980000,850,343,3.150255,1.128666
Harvey,81,1283,0,1344,523.752222,1159,400,3.507269,1.159970
Lil Trump,78,1290,0,1303,501.113333,1118,362,3.461259,1.135840
Lima Time,72,1254,40,1240,521.120000,1162,384,3.782323,1.246774
Midnight,70,1207,87,1189,420.143333,974,330,3.180227,1.096720
Roiders,56,875,64,863,336.916667,749,273,3.513615,1.184241
Trouble,73,1217,112,1207,424.011111,998,334,3.161640,1.103563


In [360]:
driver.quit()